In [ ]:
import os
os.getcwd()

In [1]:
import glob
import re

def process_mail(filename):
    result = ""
    with open(filename, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            if line.startswith("Event Invitation"):
                return None
        
            if (not re.match("(Subject|From|CC|Date|To|On|Attachments|Sent|Phone|Mobile|" + 
                             ">|--|\n|Кому>|Копия>|Тема>|Тел.?|тел\.|Моб.?|доб\.|С уважением" + 
                             "|\(See attac|Event Invitation Title|Organizer|Description|Comment|Attendees)" +
                             "|(Дубовиков Кирилл\n[а-яА-Я]\n)", line)) and (len(line) > 0) and ("@" not in line):
                
                if ((line == "Дубовиков Кирилл\n") or
                   (re.match("(Original[\s]+Message)", line)) or
                   (line.startswith("С уважением")) or
                   ("See attached file" in line)):
                    break
                    
                result += line.strip('\n') + ' '
                
    return result

emails = ''
files = glob.glob("./Sent_20160708-1921/messages/*")
files += glob.glob("./Sent Items_20160708-1923/messages/*")
for email in files:
    print("Processing %s..." % email)
    
    processed_email = process_mail(email)
    if processed_email:
        emails += processed_email

Processing ./Sent_20160708-1921/messages/20160217-Re_Несовпадение WSDL-432.txt...
Processing ./Sent_20160708-1921/messages/20150917-Re_Пересадка-11.txt...
Processing ./Sent_20160708-1921/messages/20160115-Практика по семинару (draft)-350.txt...
Processing ./Sent_20160708-1921/messages/20160112-Второй кейс для семинара-331.txt...
Processing ./Sent_20160708-1921/messages/20160114-Re_Ошибки в отчётах сотрудников за Декабрь 2015-346.txt...
Processing ./Sent_20160708-1921/messages/20141008-СМИР МОЭК. Этап I-8.txt...
Processing ./Sent_20160708-1921/messages/20160204-Fwd_ Kifi-398.txt...
Processing ./Sent_20160708-1921/messages/20151216-BRMS-292.txt...
Processing ./Sent_20160708-1921/messages/20160412-Отпуск-561.txt...
Processing ./Sent_20160708-1921/messages/20160124-Отгул-371.txt...
Processing ./Sent_20160708-1921/messages/20151015-Re_добавить в проект человека на gitlab-84.txt...
Processing ./Sent_20160708-1921/messages/20160606-Re_GUI для анализатора-657.txt...
Processing ./Sent_20160708-

In [ ]:
re.match("(Event)", "Event Invitation Title: \t РГС.Этапность реализации Location: \t Океан When: \t Mon 24 Aug 2015 12:30 – 14:00 \t \t \t \t \t \t \t invite.ics\t1,1 KB '").group(0)

In [ ]:
emails

In [2]:
import time
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.callbacks import ModelCheckpoint
import numpy as np
import random
import sys
import os       
import theano

theano.config.openmp = True
text = emails
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# WARNING, CHANGE ME
model.load_weights('./ben.17-0.12.hdf5')

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

# checkpointer = ModelCheckpoint(filepath="ben.{epoch:02d}-{loss:.2f}.hdf5", verbose=1, save_best_only=True, monitor='loss')
# model.fit(X, y, batch_size=128, nb_epoch=60, callbacks=[checkpointer])

def generate_email(text):
    print('-' * 50)
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
generate_email(text)

Using Theano backend.
WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu0 is not available  (error: Unable to get the number of gpus available: no CUDA-capable device is detected)


corpus length: 295702
total chars: 175
nb sequences: 98554
Vectorization...
Build model...


KeyboardInterrupt: 

In [ ]:
model.save_weights('./ben-12-07-2016-00-52-28.h5')